In [ ]:
# Modelo simple  para COVID con inmunidad temporal

#Este modelo es capaz de modelar una simulacion con las siguientes caacteristicas
import numpy as np
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt



In [ ]:
# ----------------- Estados -----------------
EMPTY = 0      # espacio vacío (sin persona) sitios comunes altamente concurrido
#no representa un sitio abierto mas bien un punto en comun que puede funcionar como vector,
# como perillas, barandas,cajeros, etc
SUS = 1        # susceptible, persona comun
INF = 2        # infectado (puede contagiar)
REC = 3        # recuperado (inmune temporalmente)
DEAD = 4       # fallecido (no contagia)

In [ ]:
# ----------------- Parámetros (ajusta aquí) -----------------
SIZE = 100                   # tamaño del grid (SIZE x SIZE)
person_density = 0.50        # fracción de celdas ocupadas por personas
init_infected_frac = 0.01    # fracción inicial de infectados (sobre población)
infectious_period = 9        # días que una persona permanece infectiva
mortality_prob = 0.0085      # probabilidad de morir al terminar la infección (2%)
p_trans_no_mask = 0.035      # prob. diaria de transmisión por vecino infectado (sin mascarilla)
MASK_EFFECT = 0.5            # factor multiplicador si la persona usa mascarilla (reduce p a la mitad)
mask_coverage = 0.7          # fracción de personas que usan mascarilla (0..1). Pon 0 si no usas máscaras.
p_background = 0.003         # probabilidad de infección por día independiente (espacio / comunidad) = 3%


In [ ]:
# ------- Nueva regla: inmunidad temporal -------
immunity_duration = 28       # días que dura la inmunidad tras REC; luego vuelve a SUS


In [ ]:
# ----------------- Inicialización -----------------
np.random.seed(42)

In [ ]:
def initialize_grid(size, density=0.7, init_infected_frac=0.01, mask_cov=0.0):
    grid = np.zeros((size, size), dtype=int)       # estado
    timer = np.zeros((size, size), dtype=int)      # cuenta días infectivo (si INF) o días desde recuperación (si REC)
    uses_mask = np.zeros((size, size), dtype=bool) # si la persona usa mascarilla

    # seleccionar celdas ocupadas por personas
    indices = [(i,j) for i in range(size) for j in range(size)]
    n_people = int(size*size * density)
    chosen = np.random.choice(len(indices), size=n_people, replace=False)
    chosen_coords = [indices[k] for k in chosen]

    # poner susceptibles
    for (i,j) in chosen_coords:
        grid[i,j] = SUS

    # asignar mascarillas según cobertura
    if mask_cov > 0:
        n_mask = int(n_people * mask_cov)
        mask_idxs = np.random.choice(n_people, size=n_mask, replace=False)
        for idx in mask_idxs:
            i,j = chosen_coords[idx]
            uses_mask[i,j] = True

    # asignar algunos infectados iniciales (los dejamos INF desde el inicio)
    n_init_inf = max(1, int(n_people * init_infected_frac))
    inf_idxs = np.random.choice(n_people, size=n_init_inf, replace=False)
    for idx in inf_idxs:
        i,j = chosen_coords[idx]
        grid[i,j] = INF
        timer[i,j] = infectious_period

    return grid, timer, uses_mask


In [ ]:
# ----------------- Vecinos (Moore r=1) -----------------
def neighbors(i,j,size):
    for di in [-1,0,1]:
        for dj in [-1,0,1]:
            if di==0 and dj==0:
                continue
            ni, nj = i+di, j+dj
            if 0 <= ni < size and 0 <= nj < size:
                yield ni, nj

In [ ]:
# ----------------- Regla de actualización (con inmunidad temporal) -----------------
def update_simple_with_waning(grid, timer, uses_mask,
                  p_trans_no_mask=0.05,
                  mask_effect=0.5,
                  p_background=0.03,
                  infectious_period=7,
                  mortality_prob=0.02,
                  immunity_duration=30):
    size = grid.shape[0]
    new_grid = grid.copy()
    new_timer = timer.copy()

    for i in range(size):
        for j in range(size):
            state = grid[i,j]

            # SUSCEPTIBLE: revisar contagio por vecinos y por fondo
            if state == SUS:
                prob_no_inf = 1.0

                # contagio por vecinos infectados
                for (ni,nj) in neighbors(i,j,size):
                    if grid[ni,nj] == INF:
                        # calcular probabilidad efectiva por ese vecino
                        p = p_trans_no_mask
                        # si la persona susceptible usa mascarilla, reducir entrada
                        if uses_mask[i,j]:
                            p *= mask_effect
                        # si el vecino infectado usa mascarilla, reducir salida
                        if uses_mask[ni,nj]:
                            p *= mask_effect
                        # multiplicar probabilidad de no infectarse por este vecino
                        prob_no_inf *= (1.0 - p)
                # contagio por riesgo de fondo (espacio/comunidad)
                prob_no_inf *= (1.0 - p_background)

                p_infect = 1.0 - prob_no_inf
                if np.random.rand() < p_infect:
                    new_grid[i,j] = INF
                    new_timer[i,j] = infectious_period

            # INFECTADO: decrementar timer; si finaliza decidir REC o DEAD
            elif state == INF:
                new_timer[i,j] -= 1
                if new_timer[i,j] <= 0:
                    if np.random.rand() < mortality_prob:
                        new_grid[i,j] = DEAD
                        new_timer[i,j] = 0
                    else:
                        new_grid[i,j] = REC
                        new_timer[i,j] = 0  # reiniciar contador; ahora contará días desde recuperación

            # RECUPERADO: contar días desde recuperación; si excede immunity_duration -> volver a SUS
            elif state == REC:
                new_timer[i,j] += 1
                if new_timer[i,j] >= immunity_duration:
                    new_grid[i,j] = SUS
                    new_timer[i,j] = 0

            # DEAD o EMPTY: no cambian
            else:
                pass

    return new_grid, new_timer

In [ ]:
pip install matplotlib numpy pillow

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.animation import FuncAnimation, PillowWriter
import random

# --- 1. CONFIGURACIÓN Y CONSTANTES ---
SIZE = 50
STEPS = 120
EMPTY, SUS, INF, REC, DEAD = 0, 1, 2, 3, 4

# Parámetros de la enfermedad
PERSON_DENSITY = 0.7
INIT_INFECTED_FRAC = 0.05
PROB_CONTAGIO = 0.25
PROB_MUERTE = 0.02
TIEMPO_RECUPERACION = 12
TIEMPO_INMUNIDAD = 40  # Cuánto tarda en volver a ser Susceptible

# --- 2. FUNCIONES DE LÓGICA ---
def initialize_grid(size, density, infected_frac):
    grid = np.zeros((size, size), dtype=int)
    timer = np.zeros((size, size), dtype=int)
    for i in range(size):
        for j in range(size):
            if random.random() < density:
                if random.random() < infected_frac:
                    grid[i, j] = INF
                else:
                    grid[i, j] = SUS
    return grid, timer

def update_simulation(grid, timer):
    new_grid = grid.copy()
    new_timer = timer.copy()

    for x in range(SIZE):
        for y in range(SIZE):
            if grid[x, y] == INF:
                new_timer[x, y] += 1
                # ¿Muere o se recupera?
                if new_timer[x, y] >= TIEMPO_RECUPERACION:
                    if random.random() < PROB_MUERTE:
                        new_grid[x, y] = DEAD
                    else:
                        new_grid[x, y] = REC
                        new_timer[x, y] = 0 # Reiniciar para contar inmunidad
                else:
                    # Contagio a vecinos
                    for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
                        nx, ny = (x + dx) % SIZE, (y + dy) % SIZE
                        if grid[nx, ny] == SUS and random.random() < PROB_CONTAGIO:
                            new_grid[nx, ny] = INF

            elif grid[x, y] == REC:
                new_timer[x, y] += 1
                if new_timer[x, y] >= TIEMPO_INMUNIDAD:
                    new_grid[x, y] = SUS
                    new_timer[x, y] = 0

    return new_grid, new_timer

# --- 3. PREPARACIÓN DE LA ANIMACIÓN ---
grid, timer = initialize_grid(SIZE, PERSON_DENSITY, INIT_INFECTED_FRAC)
initial_pop = np.count_nonzero(grid != EMPTY)

fig, ax = plt.subplots(figsize=(10, 7))
colores = ['#FFFFFF', '#2ca02c', '#ff4500', '#1f77b4', '#000000']
cmap = ListedColormap(colores)

img = ax.imshow(grid, cmap=cmap, vmin=0, vmax=4, animated=True)
cbar = plt.colorbar(img, ticks=[0, 1, 2, 3, 4], pad=0.1)
cbar.ax.set_yticklabels(['Vacio', 'Susc', 'Inf', 'Recu', 'Muerto'])

# Texto con estadísticas
stats_text = ax.text(1.2, 0.5, "", transform=ax.transAxes,
                     fontsize=10, verticalalignment='center', family='monospace',
                     bbox=dict(boxstyle="round", fc="white", ec="gray", alpha=0.8))

def update(frame):
    global grid, timer

    # Conteos
    sus = np.sum(grid == SUS)
    inf = np.sum(grid == INF)
    rec = np.sum(grid == REC)
    dead = np.sum(grid == DEAD)
    vivos = sus + inf + rec

    # Cálculo seguro de porcentajes (evita NameError)
    p_sus = (sus / vivos * 100) if vivos > 0 else 0
    p_inf = (inf / vivos * 100) if vivos > 0 else 0
    p_rec = (rec / vivos * 100) if vivos > 0 else 0
    p_dead = (dead / initial_pop * 100)

    # Actualizar imagen
    img.set_array(grid)
    ax.set_title(f"Evolución Pandemia - Paso {frame}/{STEPS}", fontsize=14)

    info = (f"Población: {initial_pop}\n"
            f"------------------\n"
            f"SUS: {sus:4d} ({p_sus:>5.1f}%)\n"
            f"INF: {inf:4d} ({p_inf:>5.1f}%)\n"
            f"REC: {rec:4d} ({p_rec:>5.1f}%)\n"
            f"MOR: {dead:4d} ({p_dead:>5.1f}%)")
    stats_text.set_text(info)

    # Avanzar lógica
    grid, timer = update_simulation(grid, timer)

    return [img, stats_text]

# --- 4. GENERACIÓN DEL ARCHIVO ---
print("Procesando cuadros de la animación...")
ani = FuncAnimation(fig, update, frames=STEPS, interval=100, blit=True)

writer = PillowWriter(fps=12)
ani.save("simulacion_covid.gif", writer=writer)
plt.close()

print("¡Éxito! El archivo 'simulacion_covid.gif' ha sido creado.")